Case Study по модулю SQL
Исполнитель - Мирахмедова Парвина


**Блок 1. Создание схемы и таблиц**

In [ ]:
import json
import pandas as pd
from sqlalchemy import text

from connect import set_connection

SCHEMA = "adw_works"

def create_tables():
    try:
        # читаем содержимое файла 'queries/tables.sql'
        with open('queries/tables.sql') as f:
            tables_query = f.read()
        
        # создаём схему и таблицы
        with set_connection() as psg:
            psg.execute(text(tables_query))
            psg.commit()
        
        print("Tables created successfully")
    except Exception as e:
        print(e)

create_tables()



def read_xl(sheet_name, columns_dict):
    temp_df = pd.read_excel(
        'source/adventure_works.xlsx',
        sheet_name=sheet_name,
        usecols=columns_dict.keys()
    ).rename(columns=columns_dict)
    return temp_df

with open('columns.json') as f:
    tables_dict = json.load(f) 

def insert_to_db(temp_df, tbl_name):
    with set_connection() as psg:
        temp_df.to_sql(
            schema=SCHEMA,
            name=tbl_name,
            con=psg,
            index=False,
            if_exists='append'
        )

def etl(sheet_name, columns_dict, tbl_name):
    print(f"inserting data to {tbl_name}...")
    temp_df = read_xl(sheet_name, columns_dict)
    insert_to_db(temp_df, tbl_name)
    
for k, v in tables_dict.items():
    etl(k, v["columns"], v["table_name"])



**Блок 2. Аналитические задачи**

***Секция 1. Анализ клиентов***
Сегментация по доходу: Посчитайте средний годовой личный доход клиентов (YearlyIncome) в разбивке по роду деятельности (Occupation). Итоговая таблица должна содержать следующие поля: occupation, number_of_customers, avg_income.

In [4]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import numpy as np
import pandas as pd
import plotly.express as px

from connect import set_connection 

query ="""
SELECT 
    occupation,
    COUNT(customer_key) AS number_of_customers,
    round(AVG(yearly_income),2) AS avg_income
FROM 
    adw_works.customers
GROUP BY 
    occupation
ORDER BY 
    occupation;
"""


with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,occupation,number_of_customers,avg_income
0,Clerical,2928,30710.38
1,Management,3075,92325.20
2,Manual,2384,16451.34
3,Professional,5520,74184.78
4,Skilled Manual,4577,51715.10


**Секция 1. Анализ клиентов**
***Семейный профиль***: Посчитайте долю (в процентах) клиентов с детьми и долю клиентов без детей. Итоговая таблица должна содержать следующие поля: *has_children* (где 1 означает - имеет детей и 0 - не имеет детей), *pct_of_customer_base*.

In [8]:

query ="""SELECT 
    number_of_children,
    round(COUNT(customer_key) * 100.0 / (SELECT COUNT(*) FROM adw_works.customers),2) AS pct_of_customer_base
FROM 
    adw_works.customers
GROUP BY 
    number_of_children
ORDER BY 
    number_of_children;
"""
with set_connection() as conn:
    df = pd.read_sql(query, conn)
df

,number_of_children,pct_of_customer_base
0,0,60.14
1,1,13.31
2,2,8.92
3,3,6.51
4,4,5.89
5,5,5.23
